In [2]:
from keras.preprocessing.text import Tokenizer
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
import re
import pandas as pd
from keras.utils import to_categorical

from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Input
from keras.layers import Layer
from keras.preprocessing.sequence import pad_sequences

import tensorflow_hub as hub
import tensorflow as tf

import pickle


In [3]:
training_doc3 = """ 
Python is an interpreted, object-oriented, high-level programming language with dynamic semantics. Its high-level built in data structures, combined with dynamic typing and dynamic binding, make it very attractive for Rapid Application Development, as well as for use as a scripting or glue language to connect existing components together. Python's simple, easy to learn syntax emphasizes readability and therefore reduces the cost of program maintenance. Python supports modules and packages, which encourages program modularity and code reuse. The Python interpreter and the extensive standard library are available in source or binary form without charge for all major platforms, and can be freely distributed.

Often, programmers fall in love with Python because of the increased productivity it provides. Since there is no compilation step, the edit-test-debug cycle is incredibly fast. Debugging Python programs is easy: a bug or bad input will never cause a segmentation fault. Instead, when the interpreter discovers an error, it raises an exception. When the program doesn't catch the exception, the interpreter prints a stack trace. A source level debugger allows inspection of local and global variables, evaluation of arbitrary expressions, setting breakpoints, stepping through the code a line at a time, and so on. The debugger is written in Python itself, testifying to Python's introspective power. On the other hand, often the quickest way to debug a program is to add a few print statements to the source: the fast edit-test-debug cycle makes this simple approach very effective.

Web and Internet Development
Python offers many choices for web development:

Frameworks such as Django and Pyramid.
Micro-frameworks such as Flask and Bottle.
Advanced content management systems such as Plone and django CMS.
Python's standard library supports many Internet protocols:

HTML and XML
JSON
E-mail processing.
Support for FTP, IMAP, and other Internet protocols.
Easy-to-use socket interface.
And the Package Index has yet more libraries:

Requests, a powerful HTTP client library.
Beautiful Soup, an HTML parser that can handle all sorts of oddball HTML.
Feedparser for parsing RSS/Atom feeds.
Paramiko, implementing the SSH2 protocol.
Twisted Python, a framework for asynchronous network programming.
Scientific and Numeric
Python is widely used in scientific and numeric computing:

SciPy is a collection of packages for mathematics, science, and engineering.
Pandas is a data analysis and modeling library.
IPython is a powerful interactive shell that features easy editing and recording of a work session, and supports visualizations and parallel computing.
The Software Carpentry Course teaches basic skills for scientific computing, running bootcamps and providing open-access teaching materials.
Education
Python is a superb language for teaching programming, both at the introductory level and in more advanced courses.

Books such as How to Think Like a Computer Scientist, Python Programming: An Introduction to Computer Science, and Practical Programming.
The Education Special Interest Group is a good place to discuss teaching issues.
Desktop GUIs
The Tk GUI library is included with most binary distributions of Python.

Some toolkits that are usable on several platforms are available separately:

wxWidgets
Kivy, for writing multitouch applications.
Qt via pyqt or pyside
Platform-specific toolkits are also available:

GTK+
Microsoft Foundation Classes through the win32 extensions
Software Development
Python is often used as a support language for software developers, for build control and management, testing, and in many other ways.

SCons for build control.
Buildbot and Apache Gump for automated continuous compilation and testing.
Roundup or Trac for bug tracking and project management.
Business Applications
Python is also used to build ERP and e-commerce systems:

Odoo is an all-in-one management software that offers a range of business applications that form a complete suite of enterprise management applications.
Tryton is a three-tier high-level general purpose application platform.
"""

In [5]:
 nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:

def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

cleaned = re.sub(r'\W+', ' ', training_doc3).lower()
tokens = word_tokenize(cleaned)
train_len = 3+1
text_sequences = []
for i in range(train_len,len(tokens)):
    seq = tokens[i-train_len:i]
    text_sequences.append(seq)
sequences = {}
count = 1
for i in range(len(tokens)):
    if tokens[i] not in sequences:
        sequences[tokens[i]] = count
        count += 1
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)
sequences = tokenizer.texts_to_sequences(text_sequences) 


vocabulary_size = len(tokenizer.word_counts)+1
print(vocabulary_size)
n_sequences = np.empty([len(sequences),train_len], dtype='int32')
for i in range(len(sequences)):
    n_sequences[i] = sequences[i]

texts_list = []
for line in text_sequences:
    texts_list += line
index_list = []
for line in sequences:
    index_list += line
frame = pd.DataFrame({'text':texts_list, 'loc_index':index_list})    
frame = frame.drop_duplicates()


334


In [8]:
encoderModel = hub.load('https://tfhub.dev/google/universal-sentence-encoder-large/5')

#uncomment this instead to use from tf_hub
#encoderModel = hub.load('https://tfhub.dev/google/universal-sentence-encoder-large/5')

print ("module loaded")
def embed(input):
    return encoderModel(input)

#organize training inputs and targets
train_inputs = n_sequences[:,:-1]
train_targets = n_sequences[:,-1]
train_targets = to_categorical(train_targets, num_classes=vocabulary_size)
seq_len = train_inputs.shape[1]
train_inputs.shape

#get embeddings dictionary
vocab = list(tokenizer.word_counts.keys())
vocab_embeddings = [np.array(embed([word])) for word in vocab]
embeddings_dict = {}
for i in range(len(vocab)):
    embeddings_dict[vocab[i]] = vocab_embeddings[i]

#get embeddings matrix - this becomes the weights of the non-trainable embedding layer
embeddings_matrix = np.zeros(shape=(len(frame) + 1,512))

for i in range(len(frame)):
    embeddings_matrix[frame.iloc[i].loc_index] = embeddings_dict[frame.iloc[i].text]


#Build the Network    
embedding_layer = Embedding(len(frame) + 1, 512, weights=[embeddings_matrix], input_length=seq_len, trainable=False)
model = Sequential()
model.add(embedding_layer)
model.layers[0].trainable = False
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(50,activation='relu'))
model.add(Dense(vocabulary_size, activation='softmax'))
print(model.summary())

#Compile the Network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_inputs,train_targets,epochs=500,verbose=1)

#Save model and tokenizer
model.save("word_embedding.h5")
save_obj(tokenizer, 'tokenizer_saved')



module loaded
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 3, 512)            171008    
_________________________________________________________________
lstm (LSTM)                  (None, 3, 50)             112600    
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense (Dense)                (None, 50)                2550      
_________________________________________________________________
dense_1 (Dense)              (None, 334)               17034     
Total params: 323,392
Trainable params: 152,384
Non-trainable params: 171,008
_________________________________________________________________
None
Epoch 1/500
20/20 [==============================] - 3s 8ms/step - loss: 5.8098 - accuracy: